In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("24121003_yellowtaxi_trip_count").getOrCreate()

24/12/11 16:30:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/11 16:30:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/12/11 16:30:44 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [7]:
import os
trip_files = '/trips/*'
zone_file = 'taxi+_zone_lookup.csv'
directory = os.path.join(os.getcwd(), 'data')

In [8]:
trips_df = spark.read.csv(f'file:///{directory}/{trip_files}', inferSchema=True, header=True)

In [9]:
zone_df = spark.read.csv(f'file:///{directory}/{zone_file}', inferSchema=True, header=True)

In [23]:
zone_df.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [10]:
trips_df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [11]:
trips_df.createOrReplaceTempView('trips')
zone_df.createOrReplaceTempView('zone')

In [12]:
query = '''
select
t.VendorID,
TO_DATE(t.tpep_pickup_datetime) as pickup_date,
TO_DATE(t.tpep_dropoff_datetime) as dropoff_date,
HOUR(t.tpep_pickup_datetime)  as pickup_time,
HOUR(t.tpep_dropoff_datetime) as dropoff_time,
t.passenger_count,
t.trip_distance,
t.tip_amount,
t.total_amount,
t.payment_type,
pz.Zone as pickup_zone,
dz.Zone as dropoff_zone
from trips t
LEFT JOIN zone pz ON t.PULocationID = pz.LocationID
LEFT JOIN zone dz ON t.DOLocationID = dz.LocationID
'''

In [13]:
comb_df = spark.sql(query)

In [14]:
comb_df.count()

15000700

In [16]:
comb_df.show(5)

+--------+-----------+------------+-----------+------------+---------------+-------------+----------+------------+------------+-----------------+--------------+
|VendorID|pickup_date|dropoff_date|pickup_time|dropoff_time|passenger_count|trip_distance|tip_amount|total_amount|payment_type|      pickup_zone|  dropoff_zone|
+--------+-----------+------------+-----------+------------+---------------+-------------+----------+------------+------------+-----------------+--------------+
|       2| 2021-03-01|  2021-03-01|          0|           0|              1|          0.0|       0.0|         4.3|           2|               NV|            NV|
|       2| 2021-03-01|  2021-03-01|          0|           0|              1|          0.0|       0.0|         3.8|           2|   Manhattanville|Manhattanville|
|       2| 2021-03-01|  2021-03-01|          0|           0|              1|          0.0|       0.0|         4.8|           2|   Manhattanville|Manhattanville|
|       1| 2021-03-01|  2021-03-01

In [17]:
comb_df.createOrReplaceTempView('comb')

In [12]:
query = '''
select pickup_date, pickup_time 
from comb 
where pickup_time>0
'''

In [13]:
spark.sql(query).show()

+-----------+-----------+
|pickup_date|pickup_time|
+-----------+-----------+
| 2021-02-28|         23|
| 2021-02-28|         23|
| 2021-02-28|         23|
| 2021-02-28|         23|
| 2021-02-28|         23|
| 2021-02-28|         23|
| 2021-02-28|         23|
| 2021-03-01|         22|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
+-----------+-----------+
only showing top 20 rows



In [18]:
query = '''
select pickup_date, pickup_time 
from comb 
where pickup_date<'2020-12-31'
'''
spark.sql(query).show()

+-----------+-----------+
|pickup_date|pickup_time|
+-----------+-----------+
| 2009-01-01|          0|
| 2008-12-31|         23|
| 2009-01-01|          0|
| 2009-01-01|          0|
| 2009-01-01|          0|
| 2009-01-01|          0|
| 2009-01-01|          0|
| 2009-01-01|          1|
| 2009-01-01|          0|
| 2008-12-31|         23|
| 2008-12-31|         23|
| 2008-12-31|         23|
| 2008-12-31|         23|
| 2009-01-01|          0|
| 2009-01-01|          0|
| 2009-01-01|          0|
| 2009-01-01|         16|
| 2009-01-01|         16|
| 2009-01-01|          0|
| 2009-01-01|          0|
+-----------+-----------+
only showing top 20 rows



In [19]:
comb_df.describe()

DataFrame[summary: string, VendorID: string, pickup_time: string, dropoff_time: string, passenger_count: string, trip_distance: string, tip_amount: string, total_amount: string, payment_type: string, pickup_zone: string, dropoff_zone: string]

In [20]:
spark.sql(query).explain()

== Physical Plan ==
*(3) Project [cast(tpep_pickup_datetime#41 as date) AS pickup_date#100, hour(cast(tpep_pickup_datetime#41 as timestamp), Some(Asia/Seoul)) AS pickup_time#102]
+- *(3) BroadcastHashJoin [DOLocationID#48], [LocationID#106], LeftOuter, BuildRight, false
   :- *(3) Project [tpep_pickup_datetime#41, DOLocationID#48]
   :  +- *(3) BroadcastHashJoin [PULocationID#47], [LocationID#92], LeftOuter, BuildRight, false
   :     :- *(3) Filter (isnotnull(tpep_pickup_datetime#41) AND (cast(tpep_pickup_datetime#41 as date) < 18627))
   :     :  +- FileScan csv [tpep_pickup_datetime#41,PULocationID#47,DOLocationID#48] Batched: false, DataFilters: [isnotnull(tpep_pickup_datetime#41), (cast(tpep_pickup_datetime#41 as date) < 18627)], Format: CSV, Location: InMemoryFileIndex[file:/home/lab12/src/data/trips/yellow_tripdata_2021-01.csv, file:/home/lab12/s..., PartitionFilters: [], PushedFilters: [IsNotNull(tpep_pickup_datetime)], ReadSchema: struct<tpep_pickup_datetime:string,PULocationI

In [21]:
#실행계획, 실행결과(4040)

query2 = '''
select pickup_date, pickup_time 
from comb 
where pickup_time > 0 and pickup_time<=12
'''
spark.sql(query2).explain()

== Physical Plan ==
*(3) Project [cast(tpep_pickup_datetime#41 as date) AS pickup_date#100, hour(cast(tpep_pickup_datetime#41 as timestamp), Some(Asia/Seoul)) AS pickup_time#102]
+- *(3) BroadcastHashJoin [DOLocationID#48], [LocationID#106], LeftOuter, BuildRight, false
   :- *(3) Project [tpep_pickup_datetime#41, DOLocationID#48]
   :  +- *(3) BroadcastHashJoin [PULocationID#47], [LocationID#92], LeftOuter, BuildRight, false
   :     :- *(3) Filter ((isnotnull(tpep_pickup_datetime#41) AND (hour(cast(tpep_pickup_datetime#41 as timestamp), Some(Asia/Seoul)) > 0)) AND (hour(cast(tpep_pickup_datetime#41 as timestamp), Some(Asia/Seoul)) <= 12))
   :     :  +- FileScan csv [tpep_pickup_datetime#41,PULocationID#47,DOLocationID#48] Batched: false, DataFilters: [isnotnull(tpep_pickup_datetime#41), (hour(cast(tpep_pickup_datetime#41 as timestamp), Some(Asia/..., Format: CSV, Location: InMemoryFileIndex[file:/home/lab12/src/data/trips/yellow_tripdata_2021-01.csv, file:/home/lab12/s..., Partition

In [22]:
query3 = '''
select pickup_date , count(*) as trip_count
from comb 
where pickup_time > 0
group by pickup_date
order by pickup_date
'''
spark.sql(query3).explain()

== Physical Plan ==
*(5) Sort [pickup_date#100 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(pickup_date#100 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#816]
   +- *(4) HashAggregate(keys=[pickup_date#100], functions=[count(1)])
      +- Exchange hashpartitioning(pickup_date#100, 200), ENSURE_REQUIREMENTS, [id=#812]
         +- *(3) HashAggregate(keys=[pickup_date#100], functions=[partial_count(1)])
            +- *(3) Project [cast(tpep_pickup_datetime#41 as date) AS pickup_date#100]
               +- *(3) BroadcastHashJoin [DOLocationID#48], [LocationID#106], LeftOuter, BuildRight, false
                  :- *(3) Project [tpep_pickup_datetime#41, DOLocationID#48]
                  :  +- *(3) BroadcastHashJoin [PULocationID#47], [LocationID#92], LeftOuter, BuildRight, false
                  :     :- *(3) Filter (isnotnull(tpep_pickup_datetime#41) AND (hour(cast(tpep_pickup_datetime#41 as timestamp), Some(Asia/Seoul)) > 0))
                  :     :  +- FileScan csv [tp

In [ ]:
#1.운행 거리와 요금의 상관 관계 분석

In [26]:
#2. 피크 시간대 요금 분석
from pyspark.sql.functions import hour, avg, col, format_number

# 1. Pickup 시간의 시간대(hour) 추출
trips_with_time = trips_df.withColumn("pickup_hour", hour(col("tpep_pickup_datetime")))

# 2. 시간대별 평균 요금 계산
fare_analysis_by_hour = trips_with_time.groupBy("pickup_hour") \
    .agg(
        avg("fare_amount").alias("avg_fare"),
        avg("tip_amount").alias("avg_tip"),
        avg("total_amount").alias("avg_total")
    ) \
    .withColumn("avg_fare", format_number("avg_fare", 2)) \
    .withColumn("avg_tip", format_number("avg_tip", 2)) \
    .withColumn("avg_total", format_number("avg_total", 2)) \
    .orderBy("pickup_hour")

# 3. 결과 출력
fare_analysis_by_hour.show(24)  # 24시간 시간대를 출력



+-----------+--------+-------+---------+
|pickup_hour|avg_fare|avg_tip|avg_total|
+-----------+--------+-------+---------+
|          0|   15.07|   2.49|    21.43|
|          1|   14.67|   2.39|    20.84|
|          2|   14.35|   2.27|    20.36|
|          3|   15.06|   2.25|    21.07|
|          4|   19.03|   2.40|    25.54|
|          5|   19.81|   2.42|    26.55|
|          6|   15.43|   2.16|    21.33|
|          7|   13.29|   2.06|    18.84|
|          8|   12.34|   2.05|    17.79|
|          9|   12.21|   1.98|    17.59|
|         10|   12.21|   1.93|    17.54|
|         11|   12.15|   1.93|    17.47|
|         12|   12.61|   1.95|    17.96|
|         13|   12.75|   1.99|    18.15|
|         14|   12.71|   2.05|    18.18|
|         15|   13.05|   2.11|    18.63|
|         16|   13.30|   2.24|    19.71|
|         17|   12.88|   2.26|    19.29|
|         18|   12.21|   2.25|    18.58|
|         19|   12.03|   2.22|    18.36|
|         20|   12.43|   2.24|    18.54|
|         21|   

In [27]:
from pyspark.sql.functions import hour

# 1. Pickup 시간의 시간대(hour) 추출
trips_with_time = trips_df.withColumn("pickup_hour", hour(col("tpep_pickup_datetime")))

# 2. TempView로 등록
trips_with_time.createOrReplaceTempView("trips")

# 3. SQL 쿼리 실행
fare_analysis_sql = spark.sql("""
    SELECT 
        pickup_hour,
        ROUND(AVG(fare_amount), 2) AS avg_fare,
        ROUND(AVG(tip_amount), 2) AS avg_tip,
        ROUND(AVG(total_amount), 2) AS avg_total
    FROM trips
    GROUP BY pickup_hour
    ORDER BY pickup_hour
""")

# 4. 결과 출력
fare_analysis_sql.show(24)


+-----------+--------+-------+---------+
|pickup_hour|avg_fare|avg_tip|avg_total|
+-----------+--------+-------+---------+
|          0|   15.07|   2.49|    21.43|
|          1|   14.67|   2.39|    20.84|
|          2|   14.35|   2.27|    20.36|
|          3|   15.06|   2.25|    21.07|
|          4|   19.03|    2.4|    25.54|
|          5|   19.81|   2.42|    26.55|
|          6|   15.43|   2.16|    21.33|
|          7|   13.29|   2.06|    18.84|
|          8|   12.34|   2.05|    17.79|
|          9|   12.21|   1.98|    17.59|
|         10|   12.21|   1.93|    17.54|
|         11|   12.15|   1.93|    17.47|
|         12|   12.61|   1.95|    17.96|
|         13|   12.75|   1.99|    18.15|
|         14|   12.71|   2.05|    18.18|
|         15|   13.05|   2.11|    18.63|
|         16|    13.3|   2.24|    19.71|
|         17|   12.88|   2.26|    19.29|
|         18|   12.21|   2.25|    18.58|
|         19|   12.03|   2.22|    18.36|
|         20|   12.43|   2.24|    18.54|
|         21|   

In [28]:
#2.1 가장 요금이 비싼 시간대 탑3 추출
# Top 3 시간대 추출 쿼리 실행
top3_expensive_hours = spark.sql("""
    SELECT 
        pickup_hour,
        ROUND(AVG(fare_amount), 2) AS avg_fare,
        ROUND(AVG(tip_amount), 2) AS avg_tip,
        ROUND(AVG(total_amount), 2) AS avg_total
    FROM trips
    GROUP BY pickup_hour
    ORDER BY avg_total DESC
    LIMIT 3
""")

# 결과 출력
top3_expensive_hours.show()


+-----------+--------+-------+---------+
|pickup_hour|avg_fare|avg_tip|avg_total|
+-----------+--------+-------+---------+
|          5|   19.81|   2.42|    26.55|
|          4|   19.03|    2.4|    25.54|
|          0|   15.07|   2.49|    21.43|
+-----------+--------+-------+---------+



In [ ]:
#3. 지불 유형별 요금 , 팁 분석

In [ ]:
#4. 승차지역/ 하차지역별 평균거리, 요금

In [ ]:
#5. 팁의 비율에 따라 거리, 여행 건수 서비스 관련 분석

In [29]:
spark.stop()